In [1]:
%%capture
!pip install numpy pandas statmodels scipy

In [2]:
import numpy as np
import pandas as pd

from random import choices
from scipy import stats
from statsmodels.api import OLS

# An Implementation of the X-Learner

By Alessio Zanga and Fabio Stella

## Implementation

Following the pseudocode [here](https://www.pnas.org/doi/suppl/10.1073/pnas.1804597116/suppl_file/pnas.1804597116.sapp.pdf), we implement the X-Learner esitmator as *fit* and *predict* fuctions.



In [3]:
class XLearner:
    g: OLS      # Propensity score estimator.
    
    mu_0: OLS   # Estimator for Y0 ~ X0.
    mu_1: OLS   # Estimator for Y1 ~ X1.
    tau_0: OLS  # Estimator for D0 ~ X0.
    tau_1: OLS  # Estimator for D1 ~ X1.

    def fit(self, X: pd.DataFrame, y: pd.DataFrame, w: pd.DataFrame):
        # Fit the propensity score estimator.
        self.g = OLS(w, X).fit()
        
        # Compute the subset indicator.
        I0, I1 = (w == 0), (w == 1)
        # Subset the dataset.
        X0, y0, X1, y1 = X[I0], y[I0], X[I1], y[I1]
        # Estimate the response function.
        self.mu_0 = OLS(y0, X0).fit()
        self.mu_1 = OLS(y1, X1).fit()
        # Compute the treatment effects.
        D0 = self.mu_1.predict(X0) - y0
        D1 = y1 - self.mu_0.predict(X1)
        # Estimate CATE in two ways.
        self.tau_0 = OLS(D0, X0).fit()
        self.tau_1 = OLS(D1, X1).fit()

        return self

    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        # Estimate the propensity score.
        g_hat = self.g.predict(X)

        # Average the estimates.
        tau_hat =  g_hat * self.tau_0.predict(X) + (1 - g_hat) * self.tau_1.predict(X)

        return pd.DataFrame(tau_hat, columns = ["tau"])

In [4]:
# Read train data.
train = pd.read_csv("assignment_2_train.csv")
train_true_cate = pd.read_csv("assignment_2_train_true_cate.csv")
# Read test data.
test = pd.read_csv("assignment_2_test.csv")
test_true_cate = pd.read_csv("assignment_2_test_true_cate.csv")

In [5]:
train

,W,Y,X1,X2,X3,X4,X5
0,1,7.228835,-0.034232,0.137905,0.116655,0.184476,1.014579
1,0,-15.685355,-0.822340,0.152665,-0.311184,1.583870,0.222011
2,0,-13.051963,0.236883,-0.419268,-1.378128,-1.494821,-1.157545
3,0,-97.793240,-1.014313,-1.324873,-1.479687,1.058128,-0.371527
4,0,-16.078041,0.595424,-0.690058,-1.102669,0.594276,-1.681140
...,...,...,...,...,...,...,...
99995,1,-97.090330,-0.544242,-1.456208,0.064727,-2.707226,0.341037
99996,1,75.350434,0.937368,0.827109,-0.468205,-1.480719,0.971605
99997,0,116.975403,1.616996,1.375534,1.585207,1.683143,0.863298
99998,1,-130.297218,-1.729727,-1.331081,-0.905050,-0.477871,-1.030475


In [6]:
train_true_cate

,tau
0,0.586830
1,-1.703695
2,-1.385692
3,-9.667301
4,-1.664020
...,...
99995,-8.913764
99996,6.947649
99997,11.728656
99998,-11.844586


In [7]:
# Set X, y, w names.
X, y, w = ["X1", "X2", "X3", "X4", "X5"], "Y", "W"

In [8]:
# Fit a new X-Learner estimator.
x_learner = XLearner().fit(train[X], train[y], train[w])

In [9]:
# Estimate CATE over the train set.
train_pred_cate = x_learner.predict(train[X])
train_pred_cate

,tau
0,0.588896
1,-1.701724
2,-1.368806
3,-9.671761
4,-1.658477
...,...
99995,-8.937206
99996,6.977930
99997,11.732404
99998,-11.861557


In [10]:
# Estimate CATE over the test set.
test_pred_cate = x_learner.predict(test[X])
test_pred_cate

,tau
0,-1.872533
1,6.998673
2,-0.485297
3,-1.009029
4,5.412140
...,...
995,-3.059239
996,-1.571680
997,-2.934573
998,-4.551591


In [11]:
# Compute the mean relative error over the train set ...
train_rel_err = float(((train_pred_cate - train_true_cate) / train_true_cate).abs().mean() * 100)
# ... and the test set.
test_rel_err  = float((( test_pred_cate - test_true_cate ) / test_true_cate ).abs().mean() * 100)

print(f"CATE train relative error is {train_rel_err:.4f}%, while CATE test relative error is {test_rel_err:.4f}%.")

CATE train relative error is 1.5991%, while CATE test relative error is 1.5908%.


Next, the implementation of the *compute_ci* procedure for a specific point *p*. It is trivial to generalize to more than just one point.

In [12]:
def compute_ci(
    X: pd.DataFrame,
    y: pd.DataFrame,
    w: pd.DataFrame,
    p: pd.DataFrame,
    B: int,
    alpha: float
):
    # Get indices stratified on treatment.
    S0, S1 = w.index[w == 0].tolist(), w.index[w == 1].tolist()
    # Get numbers of samples per strata.
    n0, n1 = len(S0), len(S1)
    # List of bootstrap estimates.
    tau_hat_b = []
    # Perform the bootstrap.
    for b in range(B):
        # Resample dataset with replacement.
        s_b = choices(S0, k = n0) + choices(S1, k = n1)
        X_b, y_b, w_b = X.iloc[s_b, :], y.iloc[s_b], w.iloc[s_b]
        # Fit the estimator on the bootstrap.
        tau_hat_b.append(XLearner().fit(X_b, y_b, w_b).predict(p))
    # Fit the estimator on the data set.
    tau_hat = XLearner().fit(X, y, w).predict(p)
    # Compute the standard deviation of the boostrap estimates.
    sigma = np.std(pd.concat(tau_hat_b))
    # Compute the confidence interval with the given signficance level alpha.
    lower, upper = stats.norm.interval(1 - alpha, loc = tau_hat, scale = sigma)

    return (float(tau_hat.to_numpy()), (float(lower), float(upper)))

In [13]:
%%time
compute_ci(
    X = train[X],
    y = train[y],
    w = train[w],
    p = test.iloc[[0], :][X],
    B = 100,
    alpha = 0.05
)

CPU times: user 32.2 s, sys: 20.4 s, total: 52.6 s
Wall time: 37.8 s


(-1.8725330791003059, (-1.8948436763157277, -1.850222481884884))